In [45]:
# Note that E represents the encoding that we are testing it out for. We know that for |E| <= 5, the statement holds by raw computation.
# What about when |E| >= 6? The purpose here is to generate different permutations of E such that the root is included.
# We will then generate E' with E and also run computations with E' such that we get different coefficients in the polynomials.

In [46]:
# Note that we have the current recurrence relation to determine the number of subtrees obtained from a given encoding as follows:
# F(n,e) = 0, if n = 0
# F(n,e) = 2F(n - 2, e[1:]), if e[0] = 1
# F(n,e) = F(n - 1, e[1:]), if e[0] = 0
# Then, we can turn that recurrence relation into a generating function polynomial given by:
# P_{n,e}(x)= \prod_{i=1}^{n} [(1-e_i)*( P_{n-i}(x) )+(e_i)*x*[P_{n-(i+1)}(x)]^2]
# Where the end levels of the base case is: P_{-1} =P_{0} = 1 for computations on last levels (i=n, n-1).

R.<x> = PolynomialRing(QQ)

# Base cases for P(-1) and P(0)
def P(n, e):
    if n == -1 or n == 0:
        return R(1)
    
    result = R(1)
    for i in range(1, n + 1):
        term1 = (1 - e[i-1]) * P(n - i, e) # Recursive call #1
        term2 = e[i-1] * x * (P(n - (i + 1), e))^2 # Recursive call #2
        result *= (term1 + term2)
    
    return result

In [47]:
def generate_encodings(n, current_encoding=None, encodings=None):
    if current_encoding is None:
        current_encoding = [1] 
    
    if encodings is None:
        encodings = [] 
    
    if len(current_encoding) == n: 
        encodings.append(current_encoding)
        return encodings
    
    last_bit = current_encoding[-1]
    
    # If the last bit is 1, the next bit must be 0
    if last_bit == 1:
        generate_encodings(n, current_encoding + [0], encodings)
    # If the last bit is 0, the next bit can be either 0 or 1
    else:
        generate_encodings(n, current_encoding + [0], encodings)
        generate_encodings(n, current_encoding + [1], encodings)
    
    return encodings



In [48]:
n = 6
valid_encodings = generate_encodings(n)
print(valid_encodings)

[[1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 1], [1, 0, 0, 0, 1, 0], [1, 0, 0, 1, 0, 0], [1, 0, 0, 1, 0, 1], [1, 0, 1, 0, 0, 0], [1, 0, 1, 0, 0, 1], [1, 0, 1, 0, 1, 0]]


In [49]:
# We will then calculate the augmented polynomial given by the equation:
# P_{n,e}(x)= \prod_{i=1}^{n} [(1-e_i)*( P_{n-i}(x) )+(e_i)*x*[P_{n-(i+1)}(x)]^2]
# We will then check for E and E' and hopefully prove that our hypothesis that P_{n,E'}(x) - P_{n,E}(x) >= 0

n = 4
valid_encodings = generate_encodings(n)

for e in valid_encodings:
    print(f"Encoding: {e}")
    print(P(n, e))
    print(f"Encoding: {e[::-1]}")
    print(P(n, e[::-1]))

Encoding: [1, 0, 0, 0]
x^5
Encoding: [0, 0, 0, 1]
x
Encoding: [1, 0, 0, 1]
x^6
Encoding: [1, 0, 0, 1]
x^6
Encoding: [1, 0, 1, 0]
x^5
Encoding: [0, 1, 0, 1]
x^4
